# Load `hdf5` file directly using memory mapping

In [ ]:
import h5py

import mmap

import numpy as np

def metadata_to_array(metadata, mapping):
    """Turn array metadata into a NumPy array."""
    shape = metadata['shape']
    dtype = metadata['dtype']
    offset = metadata['offset']
    length = np.prod(shape)

    return np.frombuffer(mapping, dtype=dtype, count=length, offset=offset).reshape(shape)

def extract_arrays_metadata(hdf5_path, print_metadata=False):
    """Extract arrays metadata from an HDF5 file."""
    arrays_metadata = {}

    with h5py.File(hdf5_path, 'r') as fobj:
        def dump(name, item):
            if isinstance(item, h5py.Dataset):
                if print_metadata:
                    print(name, item.shape, item.dtype)

                arrays_metadata[name] = dict(
                    offset=item.id.get_offset(), 
                    shape=item.shape, 
                    dtype=item.dtype,
                    filename=hdf5_path
                )

        fobj.visititems(dump)

    return arrays_metadata

def extract_array(arrays_metadata, array_name):
    """Extract NumPy array from an HDF5 file, given the arrays metadata and the array name you want to extract."""
    metadata = arrays_metadata[array_name]
    
    with open(metadata['filename'], 'rb') as fobj:
        mapping = mmap.mmap(fobj.fileno(), 0, access=mmap.ACCESS_READ)

        return metadata_to_array(metadata, mapping)

In [ ]:
arrays_metadata = extract_arrays_metadata('marin0.h5')

vertices = extract_array(arrays_metadata, 'nodesSpatial_Domain0')
tetrahedron_indices = extract_array(arrays_metadata, 'elementsSpatial_Domain0')
phi = extract_array(arrays_metadata, 'phi0')

In [ ]:
from pythreejs import BoxGeometry, Mesh, MeshStandardMaterial, Side

from ipygany import Scene, Data, Component, TetraMesh, Alpha, IsoSurface, Threshold, IsoColor

In [ ]:
box = Mesh(
    geometry=BoxGeometry(width=0.159, height=0.399, depth=0.159), 
    material=MeshStandardMaterial(color='lightgreen'),
    position=(2.47, 0.5, 0.08),
)

boundaries = Mesh(
    geometry=BoxGeometry(width=3.202, height=1.01, depth=1.01), 
    material=MeshStandardMaterial(color='lightgray', side=Side.BackSide),
    position=(1.61, 0.5, 0.5),
)

In [ ]:
phi_component = Component(name='Phi', array=phi.flatten())

mesh = TetraMesh(
    vertices=vertices,
    tetrahedron_indices=tetrahedron_indices,
    data=[Data(name='Phi', components=[phi_component])],
    environment_meshes=[box, boundaries]
)

surface = Threshold(mesh, input='Phi', min=-100, max=0, dynamic=True)
# surface = IsoSurface(mesh, input='Phi', value=0., dynamic=True)

water = Alpha(surface, input=0.7)

scene = Scene((water, ))
scene.background_color = 'aliceblue'

scene

In [ ]:
phi_component.array = extract_array(arrays_metadata, 'phi62').flatten()

In [ ]:
from time import sleep

for i in range(77):
    phi_component.array = extract_array(arrays_metadata, 'phi{}'.format(i)).flatten()

    sleep(0.05)